# Báo cáo Project
Lớp TTNT-156272, Nhóm N04

## 1. Thông tin chung

### Thành viên
- Nguyễn Tuấn Thành 20235833
- Lương Hoàng Vinh 20235873
- Nguyễn Minh An 20235632
- Trần Quốc Duy 20207643

### Lịch thực hiện
- W25: Đăng ký nhóm 
- W26: Đề xuất project (1/3)
- W31: Báo cáo tiến độ giữa kỳ (5/4)
- W37: Hoàn thành và gửi báo cáo project (17/5)
- W38-40: Trình bày project, Q&A

## 2. Đề xuất project (W26)

### Bài toán
Bài toán: Hệ thống giao thông công cộng thông minh
Mô tả bài toán
Một thành phố có nhiều tuyến xe buýt và tàu điện. Mỗi tuyến có các trạm dừng cố định, mỗi trạm kết nối với các tuyến khác nhau. Người dùng muốn tìm lộ trình nhanh nhất hoặc ít đổi tuyến nhất từ một trạm bất kỳ đến trạm đích.

Yêu cầu
Dữ liệu đầu vào:

Danh sách các tuyến xe buýt hoặc tàu điện, mỗi tuyến gồm một danh sách các trạm theo thứ tự.
Thời gian di chuyển giữa các trạm.
Danh sách các điểm giao nhau giữa các tuyến (trạm trung chuyển).
Điểm xuất phát S và điểm đích D.
Dữ liệu đầu ra:

Lộ trình tối ưu dựa theo thời gian ngắn nhất hoặc ít đổi tuyến nhất.
Tổng thời gian hoặc tổng số lần đổi tuyến.

### Phương pháp
Mô hình hóa bằng đồ thị, trong đó:

Đỉnh là các trạm,

Cạnh là kết nối giữa các trạm (có trọng số là thời gian di chuyển).

Tìm đường ngắn nhất bằng Dijkstra (nếu chỉ xét thời gian) hoặc BFS (nếu chỉ xét số lần đổi tuyến).
Xây dựng heuristic nếu dùng A* (ưu tiên tuyến có ít lần đổi chuyến).

### Phân công
- NM An: code, thuyết trình
- NT Thành: code, thuyết trình
- LH Vinh: code, thuyết trình
- TQ Duy: code, thuyết trình

## 3. Tiến độ giữa kỳ (W31)

### Chương trình

In [ ]:
# -*- coding: utf-8 -*-
# === KHỐI 1: IMPORTS VÀ HẰNG SỐ ===

# Import các thư viện cần thiết
import networkx as nx
import itertools
import math
import copy
import time # Để đo thời gian thực thi
from collections import deque # Để triển khai BFS

# --- Các hằng số cấu hình ---
# Thời gian ước tính cho việc chuyển tuyến (phút)
TRANSFER_TIME_PENALTY = 5

# Chi phí (số lần) cho một lần chuyển tuyến khi tối ưu theo số lần chuyển
TRANSFER_COST = 1

print("Khối 1: Imports và Hằng số đã được nạp.")

In [ ]:
# === KHỐI 2: ĐỊNH NGHĨA DỮ LIỆU BAN ĐẦU VÀ BIẾN TRẠNG THÁI ===

# --- Dữ liệu mạng lưới giao thông ban đầu ---
initial_lines = {
    'Bus 01': ['A', 'B', 'C', 'D'],
    'Bus 02': ['E', 'B', 'F', 'G'],
    'Metro M1': ['H', 'C', 'F', 'I'],
    'Tram T1': ['A', 'J', 'F', 'K']
}

initial_travel_times = {
    # Bus 01
    ('A', 'B', 'Bus 01'): 5, ('B', 'C', 'Bus 01'): 4, ('C', 'D', 'Bus 01'): 6,
    # Bus 02
    ('E', 'B', 'Bus 02'): 7, ('B', 'F', 'Bus 02'): 5, ('F', 'G', 'Bus 02'): 4,
    # Metro M1
    ('H', 'C', 'Metro M1'): 8, ('C', 'F', 'Metro M1'): 6, ('F', 'I', 'Metro M1'): 5,
    # Tram T1
    ('A', 'J', 'Tram T1'): 3, ('J', 'F', 'Tram T1'): 7, ('F', 'K', 'Tram T1'): 4,
}

# --- Các biến lưu trạng thái hiện tại của mạng lưới ---
# Sao chép dữ liệu ban đầu để có thể thay đổi
current_lines = copy.deepcopy(initial_lines)
current_travel_times = copy.deepcopy(initial_travel_times)

# Biến toàn cục cho đồ thị và mapping (sẽ được cập nhật)
# Khởi tạo rỗng, sẽ được tạo/cập nhật bởi build_transport_graph
G = nx.Graph()
stops_mapping = {}

print("Khối 2: Dữ liệu ban đầu và biến trạng thái đã được khởi tạo.")

In [ ]:
# === KHỐI 3: HÀM XÂY DỰNG ĐỒ THỊ ===

def build_transport_graph(lines_data, times_data, transfer_penalty, transfer_cost):
    """
    Xây dựng hoặc xây dựng lại đồ thị mạng lưới giao thông từ dữ liệu hiện tại.

    Nodes: tuple (stop_name, line_id)
    Edges:
        - Travel Edges: weight='time', weight='transfers'=0
        - Transfer Edges: weight='time'=transfer_penalty, weight='transfers'=transfer_cost
    Returns:
        tuple: (nx.Graph, dict) - Đồ thị và mapping trạm <-> tuyến.
    """
    graph_obj = nx.Graph() # Tạo đồ thị mới mỗi lần gọi
    stops_on_lines = {}

    # Thêm các đỉnh và cạnh di chuyển (Travel Edges)
    for line_id, stops in lines_data.items():
        if len(stops) < 2: continue
        for i in range(len(stops) - 1):
            stop1, stop2 = stops[i], stops[i+1]
            node1, node2 = (stop1, line_id), (stop2, line_id)

            # Cập nhật thông tin các tuyến đi qua mỗi trạm
            for stop, node in [(stop1, node1), (stop2, node2)]:
                 if stop not in stops_on_lines: stops_on_lines[stop] = set()
                 stops_on_lines[stop].add(line_id)
                 if node not in graph_obj: graph_obj.add_node(node) # Thêm node nếu chưa có

            # Tìm thời gian di chuyển (giả sử 2 chiều như nhau)
            time_val = math.inf
            key1, key2 = (stop1, stop2, line_id), (stop2, stop1, line_id)
            if key1 in times_data: time_val = times_data[key1]
            elif key2 in times_data: time_val = times_data[key2]

            # Thêm cạnh di chuyển nếu chưa tồn tại
            if not graph_obj.has_edge(node1, node2):
                 graph_obj.add_edge(node1, node2, time=time_val, transfers=0)

    # Thêm các cạnh chuyển tuyến (Transfer Edges)
    for stop, available_lines in stops_on_lines.items():
        if len(available_lines) > 1: # Là trạm trung chuyển
            for line1, line2 in itertools.combinations(available_lines, 2):
                node1, node2 = (stop, line1), (stop, line2)
                # Chỉ thêm cạnh nếu cả 2 node tồn tại và chưa có cạnh nối
                if node1 in graph_obj and node2 in graph_obj and not graph_obj.has_edge(node1, node2):
                     graph_obj.add_edge(node1, node2, time=transfer_penalty, transfers=transfer_cost)

    # print(f"Đồ thị được xây dựng/cập nhật với {graph_obj.number_of_nodes()} nút và {graph_obj.number_of_edges()} cạnh.")
    return graph_obj, stops_on_lines

print("Khối 3: Hàm build_transport_graph đã được định nghĩa.")

In [ ]:
# === KHỐI 4: HÀM ĐỊNH DẠNG LỘ TRÌNH VÀ TÍNH TOÁN CHỈ SỐ ===

def format_path(path, graph):
    """
    Định dạng đường đi (list of nodes) thành dạng text dễ đọc.
    Tính toán lại thời gian và số lần chuyển dựa trên các cạnh của path.
    """
    if not path:
        return "Không tìm thấy đường đi.", 0, 0

    formatted_steps = []
    final_time = 0
    final_transfers = 0
    segments = len(path) - 1

    for i in range(segments):
        u_node = path[i]     # Node hiện tại (stop_u, line_u)
        v_node = path[i+1]   # Node tiếp theo (stop_v, line_v)
        stop_u, line_u = u_node
        stop_v, line_v = v_node

        edge_data = graph.get_edge_data(u_node, v_node)
        if edge_data:
            final_time += edge_data.get('time', 0)

        is_transfer = (stop_u == stop_v and line_u != line_v)

        # Bắt đầu lộ trình
        if i == 0:
            formatted_steps.append(f"1. Đi tuyến {line_u} từ trạm '{stop_u}'")

        # Xử lý các bước tiếp theo
        if is_transfer:
            # Kết thúc đoạn trước khi chuyển
            formatted_steps[-1] += f" đến trạm '{stop_u}'."
            final_transfers += 1
            # Bắt đầu đoạn mới sau khi chuyển
            formatted_steps.append(f"{len(formatted_steps)+1}. Tại trạm '{stop_u}', chuyển sang tuyến {line_v}")
        elif i == segments -1: # Nếu là bước cuối cùng (và không phải chuyển tuyến)
             formatted_steps[-1] += f" đến trạm '{stop_v}'."
        # else: # Di chuyển bình thường trên cùng tuyến, không cần thêm text trừ khi là điểm cuối đoạn

    return "\n".join(formatted_steps), final_time, final_transfers

def calculate_path_metrics(path, graph):
    """Tính toán các chỉ số (Tổng Time, Số Transfers, Số Segments) từ một path."""
    if not path:
        return 0, 0, 0 # Time, Transfers, Segments

    total_time = 0
    total_transfers = 0
    segments = len(path) - 1

    for i in range(segments):
        u, v = path[i], path[i+1]
        edge_data = graph.get_edge_data(u, v)
        if edge_data:
            total_time += edge_data.get('time', 0)
            # Kiểm tra cạnh chuyển tuyến (cùng trạm, khác tuyến)
            if u[0] == v[0] and u[1] != v[1]:
                 total_transfers += 1
            # Hoặc dựa vào thuộc tính 'transfers' nếu tin cậy
            # total_transfers += edge_data.get('transfers', 0)

    return total_time, total_transfers, segments

print("Khối 4: Hàm format_path và calculate_path_metrics đã được định nghĩa.")

In [ ]:
# === KHỐI 5: HÀM HEURISTIC CHO A* ===

def heuristic_zero(u_node, v_node):
    """Heuristic đơn giản trả về 0. Khiến A* hoạt động như Dijkstra."""
    # u_node: node hiện tại (stop, line)
    # v_node: node đích (stop, line) - thường là đích cuối cùng khi gọi từ bên ngoài
    return 0

# Ví dụ về heuristic dựa trên khoảng cách (cần dict tọa độ `stops_coords`)
# def heuristic_distance(u_node, v_node):
#     stop_u, _ = u_node
#     stop_v, _ = v_node
#     # Giả sử có stops_coords = {'A': (lat, lon), ...}
#     if stop_u in stops_coords and stop_v in stops_coords:
#         dist = calculate_distance(stops_coords[stop_u], stops_coords[stop_v]) # Hàm tính khoảng cách
#         avg_speed_m_per_min = 20 * 1000 / 60 # Ví dụ tốc độ trung bình
#         return dist / avg_speed_m_per_min
#     return 0 # Default heuristic

print("Khối 5: Hàm heuristic_zero cho A* đã được định nghĩa.")

In [ ]:
# === KHỐI 6: CÁC HÀM TÌM ĐƯỜNG THEO THUẬT TOÁN ===

# --- Hàm gốc tìm đường ngắn nhất (Dùng bởi Dijkstra Wrappers) ---
def find_shortest_path(graph, start_stop, end_stop, stops_map, weight='time'):
    """Tìm đường đi ngắn nhất sử dụng thuật toán Dijkstra của NetworkX."""
    if start_stop not in stops_map or end_stop not in stops_map:
        return None, float('inf')

    # Lấy các node hợp lệ trong đồ thị tương ứng với trạm bắt đầu/kết thúc
    start_nodes = [(start_stop, line) for line in stops_map.get(start_stop, set()) if (start_stop, line) in graph]
    end_nodes_set = { (end_stop, line) for line in stops_map.get(end_stop, set()) if (end_stop, line) in graph }

    if not start_nodes or not end_nodes_set:
         return None, float('inf') # Không có node hợp lệ

    shortest_path_found = None
    min_cost_found = float('inf')

    # Chạy Dijkstra từ mỗi node bắt đầu có thể có
    for s_node in start_nodes:
        try:
            # Tìm đường đi ngắn nhất từ s_node đến *tất cả* các node khác
            lengths, paths = nx.single_source_dijkstra(graph, s_node, weight=weight)

            # Tìm đường đi ngắn nhất trong số các node kết thúc có thể có
            for e_node in end_nodes_set:
                 if e_node in lengths and lengths[e_node] < min_cost_found:
                     min_cost_found = lengths[e_node]
                     shortest_path_found = paths[e_node]
        except nx.NetworkXNoPath: continue # Không có đường từ s_node này
        except Exception: continue # Bỏ qua các lỗi khác

    return shortest_path_found, min_cost_found

# --- Wrapper cho Dijkstra ---
def find_dijkstra_path(graph, start_stop, end_stop, stops_map, weight='time'):
    """Wrapper gọi hàm tìm đường Dijkstra với trọng số cụ thể."""
    return find_shortest_path(graph, start_stop, end_stop, stops_map, weight)

# --- Hàm tìm đường A* ---
def find_astar_path(graph, start_stop, end_stop, stops_map, heuristic_func, weight='time'):
    """Tìm đường bằng A*."""
    if start_stop not in stops_map or end_stop not in stops_map:
        return None, float('inf')

    start_nodes = [(start_stop, line) for line in stops_map.get(start_stop, set()) if (start_stop, line) in graph]
    end_nodes_set = { (end_stop, line) for line in stops_map.get(end_stop, set()) if (end_stop, line) in graph }

    if not start_nodes or not end_nodes_set: return None, float('inf')

    shortest_path_found = None
    min_cost_found = float('inf')

    for s_node in start_nodes:
        for e_node in end_nodes_set: # A* cần target cụ thể
            # Định nghĩa heuristic phù hợp với API (cần u và v)
            # nhưng logic thường dựa trên u và đích cuối cùng (e_node)
            def h_for_astar(u, v):
                return heuristic_func(u, e_node) # Sử dụng đích cuối làm target cho heuristic

            try:
                path = nx.astar_path(graph, s_node, e_node, heuristic=h_for_astar, weight=weight)
                cost = nx.path_weight(graph, path, weight=weight) # Tính cost thực tế
                if cost < min_cost_found:
                    min_cost_found = cost
                    shortest_path_found = path
            except nx.NetworkXNoPath: continue
            except Exception: continue # Bỏ qua lỗi khác

    return shortest_path_found, min_cost_found

# --- Hàm tìm đường BFS ---
def find_bfs_path(graph, start_stop, end_stop, stops_map):
    """Tìm đường bằng BFS (số chặng ít nhất)."""
    if start_stop not in stops_map or end_stop not in stops_map: return None, 0

    start_nodes = [(start_stop, line) for line in stops_map.get(start_stop, set()) if (start_stop, line) in graph]
    end_nodes_set = { (end_stop, line) for line in stops_map.get(end_stop, set()) if (end_stop, line) in graph }

    if not start_nodes or not end_nodes_set: return None, 0

    queue = deque()
    visited = set()

    for s_node in start_nodes:
        queue.append((s_node, [s_node])) # (node, path_list)
        visited.add(s_node)

    while queue:
        current_node, path = queue.popleft()
        if current_node in end_nodes_set:
            return path, len(path) - 1 # Trả về path và số chặng

        for neighbor in graph.neighbors(current_node):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [neighbor]))

    return None, 0 # Không tìm thấy

print("Khối 6: Các hàm tìm đường Dijkstra, A*, BFS đã được định nghĩa.")

In [ ]:
# === KHỐI 7: HÀM SO SÁNH THUẬT TOÁN ===

def compare_algorithms(graph, start_stop, end_stop, stops_map):
    """Chạy các thuật toán và so sánh kết quả."""
    print(f"\n--- So sánh các thuật toán cho '{start_stop}' -> '{end_stop}' ---")
    results = {}

    # Định nghĩa các thuật toán cần so sánh
    algorithms_to_compare = {
        "Dijkstra (Time)": lambda: find_dijkstra_path(graph, start_stop, end_stop, stops_map, weight='time'),
        "Dijkstra (Transfers)": lambda: find_dijkstra_path(graph, start_stop, end_stop, stops_map, weight='transfers'),
        "A* (Time, h=0)": lambda: find_astar_path(graph, start_stop, end_stop, stops_map, heuristic_zero, weight='time'),
        "BFS (Segments)": lambda: find_bfs_path(graph, start_stop, end_stop, stops_map),
    }

    # In header bảng kết quả
    print("| Thuật toán             | T.gian chạy (s) | Path? | Time (phút) | Transfers | Segments |")
    print("|------------------------|------------------|-------|-------------|-----------|----------|")

    # Chạy và thu thập kết quả
    for name, func in algorithms_to_compare.items():
        start_exec_time = time.time()
        path, cost = func() # cost này là giá trị tối ưu của thuật toán đó
        end_exec_time = time.time()
        exec_time = end_exec_time - start_exec_time

        if path:
            # Tính toán các chỉ số thực tế từ path tìm được
            calc_time, calc_transfers, calc_segments = calculate_path_metrics(path, graph)
            print(f"| {name:<22} | {exec_time:<16.6f} | Yes   | {calc_time:<11.1f} | {calc_transfers:<9} | {calc_segments:<8} |")
            results[name] = {'path': path, 'metrics': (calc_time, calc_transfers, calc_segments), 'exec_time': exec_time}
        else:
            print(f"| {name:<22} | {exec_time:<16.6f} | No    | {'N/A':<11} | {'N/A':<9} | {'N/A':<8} |")
            results[name] = {'path': None, 'metrics': ('N/A', 'N/A', 'N/A'), 'exec_time': exec_time}
    print("|------------------------|------------------|-------|-------------|-----------|----------|")


    # (Tùy chọn) In ra đường đi chi tiết của mỗi thuật toán nếu muốn
    # ... (code in chi tiết như trước) ...

print("Khối 7: Hàm compare_algorithms đã được định nghĩa.")

In [ ]:
# === KHỐI 8: HÀM THÊM TUYẾN ĐƯỜNG MỚI ===

def add_new_line(lines_dict, times_dict):
    """Hàm tương tác để thêm tuyến đường mới vào dữ liệu hiện tại."""
    print("\n--- Thêm tuyến đường mới ---")
    # Nhập và kiểm tra ID tuyến
    while True:
        line_id = input("Nhập ID tuyến mới (ví dụ: Bus 03): ").strip()
        if not line_id: print("ID tuyến không được để trống."); continue
        if line_id in lines_dict: print(f"Lỗi: Tuyến '{line_id}' đã tồn tại."); continue
        break # ID hợp lệ

    # Nhập và kiểm tra chuỗi trạm
    stops_str = input("Nhập danh sách các trạm theo thứ tự, cách nhau bằng dấu phẩy: ").strip()
    stops_sequence = [s.strip() for s in stops_str.split(',') if s.strip()]
    if len(stops_sequence) < 2:
        print("Lỗi: Tuyến phải có ít nhất 2 trạm."); return False

    # Nhập thời gian cho từng đoạn
    new_line_times = {}
    print("Nhập thời gian di chuyển giữa các trạm liên tiếp:")
    all_times_entered = True
    for i in range(len(stops_sequence) - 1):
        stop1, stop2 = stops_sequence[i], stops_sequence[i+1]
        while True: # Vòng lặp nhập cho đến khi hợp lệ
            try:
                time_input = input(f"  Thời gian từ '{stop1}' đến '{stop2}' (phút): ").strip()
                time_val = float(time_input)
                if time_val <= 0: print("Thời gian phải là số dương."); continue
                # Lưu key theo định dạng chuẩn (stop1, stop2, line_id)
                new_line_times[(stop1, stop2, line_id)] = time_val
                break # Nhập thời gian hợp lệ
            except ValueError: print("Vui lòng nhập một số hợp lệ.")
            except Exception as e: print(f"Đã xảy ra lỗi: {e}"); all_times_entered = False; break
        if not all_times_entered: break # Thoát nếu có lỗi nghiêm trọng

    # Kiểm tra lại lần cuối và cập nhật dữ liệu chính
    if not all_times_entered:
        print("Thêm tuyến thất bại do lỗi nhập thời gian.")
        return False
    else:
        lines_dict[line_id] = stops_sequence
        times_dict.update(new_line_times) # Thêm thời gian của tuyến mới
        print(f"Đã thêm tuyến '{line_id}' thành công!")
        return True # Thêm thành công

print("Khối 8: Hàm add_new_line đã được định nghĩa.")

In [ ]:
# === KHỐI 9: HÀM CHÍNH ĐIỀU KHIỂN ỨNG DỤNG ===

def run_navigator():
    """Hàm chính chạy vòng lặp tương tác với người dùng."""
    # Sử dụng biến toàn cục để lưu trạng thái đồ thị và dữ liệu
    global G, stops_mapping, current_lines, current_travel_times

    while True:
        # BƯỚC QUAN TRỌNG: Xây dựng lại đồ thị từ dữ liệu hiện tại ở đầu mỗi vòng lặp
        try:
             G, stops_mapping = build_transport_graph(current_lines, current_travel_times, TRANSFER_TIME_PENALTY, TRANSFER_COST)
             if not G or not stops_mapping: # Kiểm tra cơ bản đồ thị có được tạo không
                  print("Lỗi: Không thể xây dựng đồ thị từ dữ liệu hiện tại. Vui lòng kiểm tra dữ liệu.")
                  # Có thể thêm lựa chọn reset dữ liệu ở đây
                  time.sleep(2) # Chờ chút rồi thử lại hoặc thoát
                  # return # Thoát nếu lỗi nghiêm trọng
        except Exception as build_err:
             print(f"Lỗi nghiêm trọng khi xây dựng đồ thị: {build_err}")
             print("Vui lòng kiểm tra lại dữ liệu tuyến và thời gian.")
             time.sleep(3)
             # return # Thoát nếu lỗi nghiêm trọng


        # --- Hiển thị Menu ---
        print("\n" + "="*40)
        print("--- Hệ thống định vị giao thông công cộng ---")
        print(f"Trạm: {len(stops_mapping)}, Tuyến: {len(current_lines)}")
        print("="*40)
        print("Lựa chọn:")
        print("1. Tìm đường đi (chọn thuật toán)")
        print("2. Thêm tuyến đường mới")
        print("3. So sánh các thuật toán")
        print("4. Hiển thị các trạm/tuyến hiện có")
        print("5. Thoát")
        choice = input("Nhập lựa chọn của bạn (1-5): ").strip()

        # --- Xử lý lựa chọn ---
        if choice == '1':
            # --- Tìm đường đi ---
            print("\nChọn thuật toán tìm đường:")
            print("  1. Dijkstra (Ưu tiên thời gian ngắn nhất)")
            print("  2. Dijkstra (Ưu tiên ít lần chuyển tuyến nhất)")
            print("  3. A* (Ưu tiên thời gian, heuristic=0)")
            print("  4. BFS (Ưu tiên số chặng/segment ít nhất)")
            algo_choice = input("Nhập lựa chọn thuật toán (1-4): ").strip()

            start_station = input("Nhập trạm bắt đầu: ").strip()
            end_station = input("Nhập trạm kết thúc: ").strip()

            # Kiểm tra input trạm
            if start_station not in stops_mapping or end_station not in stops_mapping:
                 print("Lỗi: Trạm bắt đầu hoặc kết thúc không tồn tại trong mạng lưới hiện tại."); continue
            if start_station == end_station: print("Trạm bắt đầu và kết thúc giống nhau."); continue

            path, cost = None, float('inf')
            algo_name = "Unknown"

            # Gọi hàm tìm đường tương ứng
            if algo_choice == '1':
                algo_name = "Dijkstra (Time)"; path, cost = find_dijkstra_path(G, start_station, end_station, stops_mapping, 'time')
            elif algo_choice == '2':
                 algo_name = "Dijkstra (Transfers)"; path, cost = find_dijkstra_path(G, start_station, end_station, stops_mapping, 'transfers')
            elif algo_choice == '3':
                 algo_name = "A* (Time, h=0)"; path, cost = find_astar_path(G, start_station, end_station, stops_mapping, heuristic_zero, 'time')
            elif algo_choice == '4':
                 algo_name = "BFS (Segments)"; path, cost = find_bfs_path(G, start_station, end_station, stops_mapping)
            else: print("Lựa chọn thuật toán không hợp lệ."); continue

            # In kết quả
            print(f"\n--- Kết quả tìm đường bằng [{algo_name}] ---")
            if path:
                route_str, time_val, transfers_val = format_path(path, G)
                segments_val = len(path) - 1
                print("Lộ trình:"); print(route_str)
                print(f"\nChỉ số: Time={time_val:.1f} phút, Transfers={transfers_val}, Segments={segments_val}")
            else: print("Không tìm thấy đường đi.")

        elif choice == '2':
            # --- Thêm tuyến mới ---
            # Hàm add_new_line tự cập nhật current_lines, current_travel_times
            add_new_line(current_lines, current_travel_times)
            # Đồ thị sẽ được xây dựng lại ở đầu vòng lặp tiếp theo

        elif choice == '3':
            # --- So sánh thuật toán ---
            start_station = input("Nhập trạm bắt đầu để so sánh: ").strip()
            end_station = input("Nhập trạm kết thúc để so sánh: ").strip()
            if start_station not in stops_mapping or end_station not in stops_mapping:
                 print("Lỗi: Trạm bắt đầu hoặc kết thúc không hợp lệ."); continue
            if start_station == end_station: print("Trạm bắt đầu và kết thúc giống nhau."); continue
            compare_algorithms(G, start_station, end_station, stops_mapping)

        elif choice == '4':
             # --- Hiển thị thông tin ---
             print("\n--- Thông tin mạng lưới hiện tại ---")
             print("\nCác trạm hiện có:", sorted(list(stops_mapping.keys())))
             print("\nCác tuyến hiện có:")
             if current_lines:
                 for line_id, stops in sorted(current_lines.items()):
                     print(f"  - {line_id}: {' -> '.join(stops)}")
             else:
                 print("  (Chưa có tuyến nào)")

        elif choice == '5':
            # --- Thoát ---
            print("Tạm biệt!")
            break # Thoát khỏi vòng lặp while True

        else:
            # --- Lựa chọn không hợp lệ ---
            print("Lựa chọn không hợp lệ. Vui lòng nhập số từ 1 đến 5.")

        # Tạm dừng để xem kết quả (trừ khi thoát)
        if choice != '5':
            input("\nNhấn Enter để tiếp tục...")

print("Khối 9: Hàm run_navigator chính đã được định nghĩa.")

In [ ]:
##### === KHỐI 10: KHỞI CHẠY ỨNG DỤNG ===

if __name__ == "__main__":
    print("\nBắt đầu chạy ứng dụng định vị...")
    # Khởi tạo đồ thị lần đầu trước khi vào vòng lặp chính
    # Điều này đảm bảo G và stops_mapping có giá trị ban đầu
    try:
        G, stops_mapping = build_transport_graph(current_lines, current_travel_times, TRANSFER_TIME_PENALTY, TRANSFER_COST)
        print("Đồ thị ban đầu đã được xây dựng.")
        run_navigator() # Bắt đầu vòng lặp chính
    except Exception as e:
        print(f"\nĐã xảy ra lỗi nghiêm trọng khi khởi chạy: {e}")
        print("Vui lòng kiểm tra lại code và dữ liệu ban đầu.")

else:
    # Nếu file này được import như một module, bạn có thể muốn
    # xây dựng đồ thị sẵn sàng để các hàm khác sử dụng.
    try:
        print("Module đang được import. Xây dựng đồ thị ban đầu...")
        G, stops_mapping = build_transport_graph(current_lines, current_travel_times, TRANSFER_TIME_PENALTY, TRANSFER_COST)
        print("Đồ thị ban đầu sẵn sàng.")
    except Exception as e:
        print(f"Lỗi khi xây dựng đồ thị ban đầu trong quá trình import module: {e}")

### Kết quả, vấn đề gặp phải
...

## 4. Cập nhật kết quả cuối kỳ (W37)

### Chi tiết phương pháp, dữ liệu 
....

### Chương trình
...

### Phân tích, đánh giá kết quả
...

### Cập nhật phân công, khối lượng công việc
<!-- công việc của các thành viên, tỷ lệ đóng góp của các thành viên -->
...